<a href="https://colab.research.google.com/github/tusker4/Sesac_Saltlux_DeepLearning/blob/main/6_NLP_Transformers_%EB%AC%B8%EC%84%9C%EC%9A%94%EC%95%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요
  - many to many
    - 대량의 말뭉치(한계) -> 소량의 말뭉치(요약)
    

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.6 MB/s eta 0:00:00


# 모델 및 토크나이저 획득 (제로샷러닝활용)
-  gpt 계열 API에 기능이 동일하게 존재
  - 여러제품들중 더 나은 모델로 사용
  - 선택에 대한 벤치마킹 필요
    - 동일 데이터, 반응시간(응답시간) 등등 체크
    

In [ ]:
# 모듈 가져오기
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [ ]:
repo = 'gogamza/kobart-summarization'

# 토크나이저 획득 => 인코딩(문장 -> 백테화), 디코딩(벡터화 -> 문장 ) 제공
tokenizer = PreTrainedTokenizerFast.from_pretrained( repo )

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
# 모델 로드
model = BartForConditionalGeneration.from_pretrained ( repo )

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# 문서요약

In [ ]:
text = '''
[헤럴드경제=유혜림 기자] 공매도 전면 금지가 시행된 지난주 포스코그룹주와 에코프로그룹주의 '빚투(빚내서 투자)'가 1100억원 넘게 늘어난 것으로 나타났다. 지난 9월부터 2차전지 관련주의 주가가 내리면서 신용잔고율도 꾸준히 감소세를 나타냈는데 다시 반등 조짐을 나타낸 것이다. 주가 하방 압력을 가했던 공매도 영향이 줄었다고 본 투심들이 ‘반짝’ 상승세에 나선 것으로 풀이된다.

'''

text = text.strip()

In [ ]:
# 인코딩
text_vec = tokenizer.encode(text)
text_vec

In [ ]:
# 요약 처리를 위해서 문장의 앞뒤에 스페셜 토큰 추가 필요
tokenizer.bos_token, tokenizer.bos_token_id, tokenizer.eos_token, tokenizer.eos_token_id, type(text_vec)

('<s>', 0, '</s>', 1)

In [ ]:
# 문장의 맨앞과 맨뒤에 0 을 추가
# 위의 예시 문장은 여러 문장이 합쳐져서 만들어진 큰 말뭉치여서 요약이 적절하지 않을 수도 있다.
input_vec = [tokenizer.bos_token_id] + text_vec + [tokenizer.bos_token_id]
print( input_vec )
## 맨앞과 맨뒤에 0 을 추가


[0, 14552, 13656, 24177, 14714, 260, 11986, 13683, 10486, 17928, 14061, 10518, 9866, 17253, 14220, 14824, 15358, 9908, 22194, 18555, 15106, 19974, 27885, 15786, 15106, 14453, 14063, 11043, 13282, 239, 11043, 9517, 11264, 14559, 240, 16388, 14460, 23661, 19538, 18603, 14173, 19743, 14130, 14141, 15558, 14150, 16411, 26272, 14342, 14453, 20623, 18446, 14125, 16362, 12063, 9102, 11991, 9866, 17836, 15665, 14896, 14547, 9526, 14304, 14461, 23277, 25185, 12007, 14547, 9519, 16746, 18372, 14042, 10788, 15112, 14428, 14039, 14622, 14061, 10518, 9866, 24391, 14390, 18731, 14285, 14233, 11477, 14108, 14143, 10773, 12354, 1543, 14919, 18207, 16297, 14173, 18902, 9908, 14130, 0]


In [ ]:
# 요약
import torch

summary_indexs = model.generate ( torch.tensor( [ input_vec ]) , max_new_tokens=40 )
print( summary_indexs )



tensor([[    2, 14061, 10518,  9866, 17253, 14220, 14824, 15358,  9908, 14140,
         18555, 15106, 19974, 27885, 15786, 15106, 14453, 14063, 11043, 13282,
         16388, 14460, 23661, 19538, 18603, 14173, 19743, 15133, 15002, 14141,
         15558, 14150, 16411, 26272, 14342, 14453, 20623, 18446, 14125, 16362,
             2]])


In [ ]:
summary_indexs.size()

torch.Size([1, 41])

In [ ]:
# 디코딩
tokenizer.decode(summary_indexs.squeeze().tolist(), skip_special_token=True)

"<usr> 공매도 전면 금지가 시행된 후 포스코그룹주와 에코프로그룹주의 '빚투'가 1100억원 넘게 늘어난 것으로 나타났는데, 이는 지난 9월부터 2차전지 관련주의 주가가 내리면서 신용<usr>"

In [ ]:
# 모듈 가져오기
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import torch

# 토크나이저, 모델 로드
repo = 'gogamza/kobart-summarization'
# 토크나이저 획득 => 인코딩(문장 -> 백테화), 디코딩(벡터화 -> 문장 ) 제공
tokenizer = PreTrainedTokenizerFast.from_pretrained( repo )
model = BartForConditionalGeneration.from_pretrained ( repo )


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:

# 함수화
def get_sentence_summary(text:str )-> str:
  # 인코딩
  text_vec = tokenizer.encode(text)
  input_vec = [tokenizer.bos_token_id] + text_vec + [tokenizer.bos_token_id] # 문장 맨앞뒤 0
  summary_indexs = model.generate ( torch.tensor( [ input_vec ]) , max_new_tokens=100 )
  # 디코딩
  summary = tokenizer.decode(summary_indexs.squeeze().tolist()[1:-1], skip_special_token=True)
  return summary

# 테스트
if __name__ == '__main__':
  while True:
    text = input('요약하고 싶은 글을 작성하세요\n')
    if text == 'z':
      break
      print(get_sentence_summary(text))




요약하고 싶은 글을 작성하세요
13일 열린 제55차 한미안보협의회(SCM)는 고도화된 북한 미사일 위협 대응을 위해 한미동맹을 과학기술동맹으로 진화시키는 내용의 이정표를 제시한 점에서 의미가 크다. 양국이 조기경보부터 요격체계까지 북한 핵·미사일 대응력 강화를 위해 공동연구 착수와 과학기술 협력을 합의하면서 방위력의 획기적 증강 및 연합작전 수행능력 제고라는 효과를 꾀할 수 있게 됐다.  신원식 국방부 장관과 로이드 오스틴 미 국방장관은 이날 용산 국방부 청사에서 열린 제55차 SCM에서 양국 국방과학기술협력을 심화하기로 합의했다. 한미 군 당국은 국방과학기술 협의체를 설치해 국방과학기술 협력 강화방안을 협의하고 있다.  눈여겨볼 대목은 고도화하고 있는 북핵·미사일에 대응하기 위한 과학기술 협력이다. 한미는 이번 SCM에서 미국 조기경보위성 정보 공유체계(SEWS)를 통해 동맹의 탐지능력을 강화하는 데 합의했다. SEWS는 우주기반의 조기경보위성을 통해 적국의 미사일 도발을 조기 탐지하고 관련 정보를 동맹·우방국에 공유하는 체계다. 한미 간 SEWS 체계가 구축된다면, 우리 입장에선 보다 정확하고 신속하게 북한의 미사일 도발 징후를 확인할 수 있다. 국방부 관계자는 "미군이 운용하고 있는 조기경보위성 정보를 실시간으로 공유받으면, 이를 우리 요격무기체계로 즉각 전파해 대응력을 높일 수 있다"고 설명했다.  북한의 미사일 발사 이전 단계부터 요격까지 단계별 필요한 기술협력 분야와 전략에 대한 연구도 이뤄진다. 한미 군 당국은 지난해 SCM에서 한미 미사일대응 정책협의체(CMWG)를 신설하고 3년간 운영하기로 합의한 바 있다. 협의체에서는 북한 신형 미사일 위협과 관련한 데이터베이스(DB)를 구축하고 시뮬레이션을 짜는 한미 미사일방어 공동연구(PAWG)가 한창이다.  두 장관은 '동맹의 포괄적 미사일대응 전략' 마련을 위한 공동연구에도 착수하기로 했다. 한미 국방부, 합동참모본부와 주한미군사령부 등 국방정책기관이 주관해 정책결정력을 높일 예정이다. 국방부 관계자는

In [ ]:
text = '''
13일 열린 제55차 한미안보협의회(SCM)는 고도화된 북한 미사일 위협 대응을 위해 한미동맹을 과학기술동맹으로 진화시키는 내용의 이정표를 제시한 점에서 의미가 크다. 양국이 조기경보부터 요격체계까지 북한 핵·미사일 대응력 강화를 위해 공동연구 착수와 과학기술 협력을 합의하면서 방위력의 획기적 증강 및 연합작전 수행능력 제고라는 효과를 꾀할 수 있게 됐다.

신원식 국방부 장관과 로이드 오스틴 미 국방장관은 이날 용산 국방부 청사에서 열린 제55차 SCM에서 양국 국방과학기술협력을 심화하기로 합의했다. 한미 군 당국은 국방과학기술 협의체를 설치해 국방과학기술 협력 강화방안을 협의하고 있다.

눈여겨볼 대목은 고도화하고 있는 북핵·미사일에 대응하기 위한 과학기술 협력이다. 한미는 이번 SCM에서 미국 조기경보위성 정보 공유체계(SEWS)를 통해 동맹의 탐지능력을 강화하는 데 합의했다. SEWS는 우주기반의 조기경보위성을 통해 적국의 미사일 도발을 조기 탐지하고 관련 정보를 동맹·우방국에 공유하는 체계다. 한미 간 SEWS 체계가 구축된다면, 우리 입장에선 보다 정확하고 신속하게 북한의 미사일 도발 징후를 확인할 수 있다. 국방부 관계자는 "미군이 운용하고 있는 조기경보위성 정보를 실시간으로 공유받으면, 이를 우리 요격무기체계로 즉각 전파해 대응력을 높일 수 있다"고 설명했다.

북한의 미사일 발사 이전 단계부터 요격까지 단계별 필요한 기술협력 분야와 전략에 대한 연구도 이뤄진다. 한미 군 당국은 지난해 SCM에서 한미 미사일대응 정책협의체(CMWG)를 신설하고 3년간 운영하기로 합의한 바 있다. 협의체에서는 북한 신형 미사일 위협과 관련한 데이터베이스(DB)를 구축하고 시뮬레이션을 짜는 한미 미사일방어 공동연구(PAWG)가 한창이다.

두 장관은 '동맹의 포괄적 미사일대응 전략' 마련을 위한 공동연구에도 착수하기로 했다. 한미 국방부, 합동참모본부와 주한미군사령부 등 국방정책기관이 주관해 정책결정력을 높일 예정이다. 국방부 관계자는 "북한 신형 미사일 위협뿐만 아니라 이를 둘러싼 복합적인 환경을 분석해 전략 및 대응전략에 반영하기 위한 것"이라고 했다.

이밖에 한미는 차세대 무선통신 기술을 연합작전에 활용하기 위해 상호운용성 확보 방안을 논의하고 무선통신 실무협의체 구성 및 운영을 위한 약정(TOR) 협의를 추진해 나가기로 했다. 국방부는 "지난 9월 한미 국방차관-국방연구공학차관 간 회담을 통해 양국 간 고위급 협의체 구축 논의를 시작하기로 했다"며 "정책적·전략적 차원의 협력을 강화해 나갈 것"이라고도 밝혔다.

'''
get_sentence_summary(text)

'한미는 지난 13일 용산 국방부 청사에서 열린 제55차 한미안보협의회(SCM)에서 고도화된 북한 미사일에 대응하기 위한 과학기술 협력을 강화하기로 합의하고 한미 국방과학기술 협의체를 설치해 국방과학기술 협력 강화방안을 협의하기로 했다.'